In [1]:
# 14.7.20
# check whether links have proper content

In [61]:
import requests
from bs4 import BeautifulSoup as bs
import pickle
import warnings
import json
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

In [3]:
with open('../data/experiment_1.pickle', 'rb') as handle:
    experiment_1 = pickle.load(handle)

In [4]:
experiment_1['failed_links']['train'][0]['url']

'http://www.newindianexpress.com/states/odisha/2014/dec/03/Education-Must-for-Tribals-689654.html'

In [4]:
requests.get('http://www.newindianexpress.com/states/odisha/2014/dec/03/Education-Must-for-Tribals-689654.html')

<Response [403]>

## The Hindu Content

In [5]:
theHindu = []

for i in experiment_1['data']:
    if "thehindu" in experiment_1['data'][i]['url']:
        theHindu.append(experiment_1['data'][i])

In [6]:
sub = []
for article in theHindu:
    if "subscription" in str(article['response'].content):
         sub.append(article)

In [7]:
sub[1]

{'response': <Response [200]>,
 'url': 'http://www.thehindu.com/todays-paper/tp-in-school/not-a-bright-move/article7793373.ece'}

In [8]:
bs(sub[1]['response'].content)

<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>
‘Not a bright move’ - IN SCHOOL - The Hindu
</title>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="‘Not a bright move’" name="title"/>
<meta content="articlepage" name="atdlayout"/>
<meta content="Heritage experts say no to illumination of the Taj" name="description"/>
<meta content="noindex" name="robots"/>
<meta content="!" name="fragment"/>
<meta content="" name="keywords"/>
<meta content="" name="news_keywords"/>
<meta content="2015-10-23T00:00:00+05:30" name="publish-date"/>
<meta content="2015-10-23T00:00:00+05:30" name="created-date"/> <meta content="2015-10-23T02:02:20+05:30" name="modified-date"/>
<meta content="PTI" name="twitter:creator"/>
<meta content="‘Not a bright move’" name="twitter:title"/>
<meta content="Heritage experts say no to illumination of the T

## SCMP and XHR

In [9]:
scmp = []

for i in experiment_1['data']:
    if "scmp" in experiment_1['data'][i]['url']:
        scmp.append(experiment_1['data'][i])

In [12]:
scmp[345]

{'response': <Response [200]>,
 'url': 'http://www.scmp.com/article/240249/ripple-effects-are-felt-following-multinational-marriages'}

['229100',
 '236261',
 '235565',
 '246661',
 '236592',
 '236091',
 '247310',
 '247972',
 '238523',
 '233873',
 '246296',
 '237326',
 '241920',
 '234465',
 '244546',
 '239150',
 '237457',
 '245441',
 '245072',
 '231030',
 '229586',
 '230873',
 '247720',
 '228679',
 '234554',
 '237427',
 '247153',
 '230757',
 '244715',
 '233288',
 '244043',
 '235662',
 '243996',
 '236383',
 '232165',
 '239194',
 '233649',
 '248592',
 '228343',
 '237770',
 '230896',
 '229140',
 '234204',
 '248305',
 '234067',
 '243060',
 '243812',
 '242026',
 '241787',
 '240908',
 '231554',
 '238526',
 '228373',
 '241048',
 '235906',
 '230566',
 '231148',
 '229281',
 '236329',
 '236769',
 '246926',
 '243211',
 '233136',
 '241724',
 '244569',
 '233784',
 '237570',
 '230754',
 '236772',
 '238418',
 '247962',
 '246755',
 '239368',
 '238013',
 '230527',
 '229507',
 '246773',
 '234126',
 '239562',
 '230226',
 '233367',
 '234800',
 '247192',
 '245573',
 '230147',
 '242884',
 '237205',
 '244738',
 '241137',
 '248342',
 '239138',

In [16]:
def get_articles(entityids):
    
    
    limit_url= "variables=%7B%22limit%22%3A10%2C%22"
    ent_url = "entityIds%22%3A%5B%22"+'%22%2C%22'.join([str(i) for i in entityids]) +"%22%5D%7D&"
    
    api_url = "https://apigw.scmp.com/content-delivery/v1?operationName=batcharticlebyid&"+limit_url+ent_url+"extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%224d7d3ce2bf773ea3b92086d8cab77f7d756917033d39985bd87ca83b7a2fc968%22%7D%7D"
    
    headers = {"apikey": "MyYvyg8M9RTaevVlcIRhN5yRIqqVssNY"}

    r = requests.get(api_url,headers=headers).json()
    
    
    return r['data']['articles']

In [31]:
for article in get_articles([entityids[3]]):
    print(get_text(article))
    break

As we are leaving Hong Kong shortly my husband and I inquired about donating excess good-quality clothes to the Salvation Army. At Wing Sing Lane we eventually found a Salvation Army office only to be told very abruptly that it was the wrong one. Staff were extremely offhand and rude. When we asked if we could leave the boxes for staff to deliver, an employee shouted at us that (and I quote) they would throw them out with the rubbish if we did. This, after we had washed and dried the clothes and taken a taxi from Sai Kung to deliver them (we have no car). When we'd staggered over to Booth Lodge the lady at the desk was very nice, but by that time we were hot, angry and late for our next appointment. Perhaps it is too much to expect a charity's staff to be charitable. I do not think General Booth would have thought so. E.L. PORTER New Territories


In [17]:
def get_text(article):
    sentences = []
    small_bodies_p= [small_body for small_body in article['body'] if small_body['type'] == 'p']
    
    for p in small_bodies_p:

        if p['type'] == 'p':
            children = p['children']
            group = [s for s in children if s['type'] in ['text','a','span']]
            
            for s in group:
                if s['type'] == "text":
                    sentences.append(s['data'].strip())
                    #print(s['data'].strip())
                else:
                    #try:
                    if len(s['children']) != 0 and'data' in s['children'][0]:
                            sentences.append(s['children'][0]['data'].strip())
                    #except:
                        #print(s)
                        #sentences.append("---------------------------")
                        #break
                    break
                    
                    
    if len(sentences[0]) ==1 :
        fs = sentences[0]+sentences[1]
        
        sentences[1] = fs
        
        sentences.pop(0)
    
    return ' '.join(sentences)

## Extract SCMP

In [53]:
entityids=[i['url'].split('www.scmp.com/article/')[1].split('/')[0] for i in scmp]
batches = [entityids[i:i+10] for i in range(len(entityids)) if i%10 == 0]

In [55]:
articles = []
for batch in batches:
    for a in get_articles(batch):
        articles.append(a)

In [57]:
texts = []

for article in articles:
    texts.append(get_text(article))

In [62]:
DATA = []
for file in ["../data/train.json","../data/test.json","../data/dev.json","../data/test_china.json"]:
    with open(file, 'r') as f:
        for line in f:
            d = json.loads(line)
            url = d['url']
            ID = d['id']
            if "scmp" in url:
                print(file)

../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_chi

../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
../data/test_china.json
